# Detecção de Acessos Não Autorizados e Insider Threats

Objetivos:

- Monitorar acessos internos a sistemas corporativos e detectar padrões suspeitos.
- Utilizar Databricks para identificar acessos fora do padrão.
- Enviar alertas por email ao time de segurança.

## 1️⃣ Gerando Arquivos de Log para Simulação

In [0]:
import random
import pandas as pd
from datetime import datetime, timedelta

# Definir usuários e recursos
usuarios = ["user_001", "user_002", "user_003", "user_004", "user_005"]
recursos = ["Banco de Dados", "Servidor Interno", "Planilha Financeira", "Relatórios Confidenciais"]
roles = {"user_001": "Admin", "user_002": "Analista", "user_003": "Analista", "user_004": "Gerente", "user_005": "TI"}


In [0]:
# Gerar dados fictícios
def gerar_logs(qtd_registros=1000):
    logs = []
    for _ in range(qtd_registros):
        user = random.choice(usuarios)
        timestamp = datetime.now() - timedelta(minutes=random.randint(1, 1440))
        ip = f"192.168.{random.randint(0,255)}.{random.randint(0,255)}"
        recurso = random.choice(recursos)
        role = roles[user]
        logs.append([user, timestamp, ip, recurso, role])
    
    df = pd.DataFrame(logs, columns=["user_id", "timestamp", "ip_address", "resource_accessed", "role"])
    
    # Ensure the directory exists
    dbutils.fs.mkdirs("dbfs:/tmp")
    
    # Add timestamp to file name
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_path = f"/dbfs/tmp/logs_acesso_simulados_{timestamp_str}.csv"
    df.to_csv(f"/dbfs/tmp/logs_acesso_simulados_{timestamp_str}.csv", index=False)
    
    return file_path, timestamp_str

In [0]:
# Gerar 1000 registros fictícios
file_path, timestamp_str = gerar_logs(1000)

# Move the file to the desired location
dbutils.fs.mv(f"dbfs:/tmp/logs_acesso_simulados_{timestamp_str}.csv", 
              f"abfss://landing@stdatasecdemo.dfs.core.windows.net/logs_acesso_simulados_{timestamp_str}.csv")

## 2️⃣ Detectando Acessos Suspeitos

In [0]:
from pyspark.sql.functions import col, hour, lit

# Carregar logs simulados
df_logs = spark.read.csv("abfss://landing@stdatasecdemo.dfs.core.windows.net/*.csv", header=True, inferSchema=True)

# Definição de regras de risco
acessos_fora_horario = (df_logs.filter((hour(col("timestamp")) < 6) | (hour(col("timestamp")) > 22))
                        .withColumn("tipo_anomalia", lit("Acesso fora do horário"))
                        )

acessos_nao_autorizados = (df_logs.filter((col("role") == "Analista") & (col("resource_accessed").like("%Confidenciais%")))
                           .withColumn("tipo_anomalia", lit("Acesso não autorizado"))
                           )

# Unindo os acessos suspeitos
acessos_suspeitos = acessos_fora_horario.union(acessos_nao_autorizados)

In [0]:
%sql
CREATE SCHEMA if not exists seguranca

In [0]:
acessos_fora_horario.write.mode("append").saveAsTable("seguranca.acessos_fora_horario")
acessos_nao_autorizados.write.mode("append").saveAsTable("seguranca.acessos_nao_autorizados")

In [0]:
%sql

CREATE TABLE IF not exists seguranca.acessos_suspeitos AS
SELECT * 
FROM seguranca.acessos_fora_horario

UNION ALL

SELECT *
FROM seguranca.acessos_nao_autorizados

## 3️⃣ Enviando Email com as Anomalias Detectadas

In [0]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

# Configuração do email
email_sender = dbutils.secrets.get(scope="key-vault-secret", key="email")
email_password = dbutils.secrets.get(scope="key-vault-secret", key="senha-email") 
email_receiver = "laismeuchi@gmail.com"


In [0]:
def enviar_email(acessos_suspeitos):
    # Construção do email
    subject = "Alerta de Acessos Suspeitos Detectados"

    # busca os acessos suspeitos para enviar
    df_acessos_supeitos = spark.sql("SELECT * FROM seguranca.acessos_suspeitos")
    body = df_acessos_supeitos.toPandas().to_string()

    msg = MIMEMultipart()
    msg["From"] = email_sender
    msg["To"] = email_receiver
    msg["Subject"] = subject
    msg.attach(MIMEText(body, "plain"))

    # Enviando o email
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(email_sender, email_password)
        server.sendmail(email_sender, email_receiver, msg.as_string())

In [0]:
if acessos_suspeitos.count() > 0:
    enviar_email(acessos_suspeitos)
else:
    print("Sem acessos suspeitos")